<a href="https://colab.research.google.com/github/lkhok22/ML-hw4/blob/main/face-recognition-DeepCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
! pip install kaggle

In [9]:
import wandb
wandb.login()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

 99% 281M/285M [00:00<00:00, 381MB/s]
100% 285M/285M [00:00<00:00, 418MB/s]


In [10]:
import pandas as pd
import numpy as np

In [11]:
train_df_t = pd.read_csv('train.csv')
test_df_t = pd.read_csv('test.csv')

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import matplotlib.pyplot as plt
import random
import os

# Set seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()


In [13]:
from PIL import Image

class FERDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = np.array(row['pixels'].split(), dtype=np.uint8).reshape(48, 48)
        label = int(row['emotion']) if 'emotion' in row else -1
        image = Image.fromarray(image)

        if self.transform:
            image = self.transform(image)

        return image, label


In [14]:
from sklearn.model_selection import train_test_split

# Split training data for validation
train_df, val_df = train_test_split(train_df_t, test_size=0.1, stratify=train_df_t['emotion'], random_state=42)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = FERDataset(train_df, transform=transform)
val_dataset = FERDataset(val_df, transform=transform)

# Use very small subset (e.g. 20 images) to test overfitting
small_train_dataset = torch.utils.data.Subset(train_dataset, list(range(20)))

small_loader = DataLoader(small_train_dataset, batch_size=4, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


In [15]:
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 48 → 24
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 24 → 12
            nn.Dropout(0.25),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),     # 12 → 6
            nn.Dropout(0.25),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 6 * 6, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 7)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [16]:
import torch

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [17]:
def train_model(model, loader, optimizer, criterion, epochs=10, use_wandb=False):
    model.to(device)  # ⬅️ Move model to GPU
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)  # ⬅️ Move data to GPU

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        acc = correct / total
        avg_loss = total_loss / len(loader)

        print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Accuracy: {acc:.4f}")

        if use_wandb:
            wandb.log({"loss": avg_loss, "accuracy": acc})


In [19]:
wandb.init(project="fer-overfit-test", name="DeepCNN-overfit-small")

model = DeepCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, small_loader, optimizer, criterion, epochs=20, use_wandb=True)

wandb.finish()


Epoch 1/20 | Loss: 6.1907 | Accuracy: 0.2500
Epoch 2/20 | Loss: 7.3802 | Accuracy: 0.2500
Epoch 3/20 | Loss: 7.2849 | Accuracy: 0.3500
Epoch 4/20 | Loss: 4.5778 | Accuracy: 0.2000
Epoch 5/20 | Loss: 4.2146 | Accuracy: 0.4500
Epoch 6/20 | Loss: 3.4279 | Accuracy: 0.4500
Epoch 7/20 | Loss: 1.5318 | Accuracy: 0.6000
Epoch 8/20 | Loss: 1.9389 | Accuracy: 0.5000
Epoch 9/20 | Loss: 0.7453 | Accuracy: 0.8000
Epoch 10/20 | Loss: 1.1070 | Accuracy: 0.6500
Epoch 11/20 | Loss: 1.1767 | Accuracy: 0.5500
Epoch 12/20 | Loss: 1.0178 | Accuracy: 0.6000
Epoch 13/20 | Loss: 0.6838 | Accuracy: 0.7500
Epoch 14/20 | Loss: 0.6681 | Accuracy: 0.7000
Epoch 15/20 | Loss: 1.0660 | Accuracy: 0.7000
Epoch 16/20 | Loss: 0.2618 | Accuracy: 0.9500
Epoch 17/20 | Loss: 1.7605 | Accuracy: 0.7500
Epoch 18/20 | Loss: 0.5499 | Accuracy: 0.8000
Epoch 19/20 | Loss: 0.5572 | Accuracy: 0.7500
Epoch 20/20 | Loss: 0.3882 | Accuracy: 0.8500


accuracy,▁▁▂▁▃▃▅▄▇▅▄▅▆▆▆█▆▇▆▇
loss,▇██▅▅▄▂▃▁▂▂▂▁▁▂▁▂▁▁▁
accuracy,0.85
loss,0.38819


In [20]:
wandb.init(project="fer-full-training", name="DeepCNN-full")

model = DeepCNN()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, optimizer, criterion, epochs=20, use_wandb=True)

wandb.finish()


Epoch 1/20 | Loss: 1.9229 | Accuracy: 0.2509
Epoch 2/20 | Loss: 1.6949 | Accuracy: 0.3150
Epoch 3/20 | Loss: 1.5673 | Accuracy: 0.3870
Epoch 4/20 | Loss: 1.4793 | Accuracy: 0.4136
Epoch 5/20 | Loss: 1.4325 | Accuracy: 0.4302
Epoch 6/20 | Loss: 1.4019 | Accuracy: 0.4423
Epoch 7/20 | Loss: 1.3678 | Accuracy: 0.4588
Epoch 8/20 | Loss: 1.3338 | Accuracy: 0.4694
Epoch 9/20 | Loss: 1.3159 | Accuracy: 0.4811
Epoch 10/20 | Loss: 1.2840 | Accuracy: 0.4926
Epoch 11/20 | Loss: 1.2636 | Accuracy: 0.5023
Epoch 12/20 | Loss: 1.2390 | Accuracy: 0.5154
Epoch 13/20 | Loss: 1.2107 | Accuracy: 0.5245
Epoch 14/20 | Loss: 1.1988 | Accuracy: 0.5288
Epoch 15/20 | Loss: 1.1828 | Accuracy: 0.5356
Epoch 16/20 | Loss: 1.1602 | Accuracy: 0.5430
Epoch 17/20 | Loss: 1.1351 | Accuracy: 0.5595
Epoch 18/20 | Loss: 1.1178 | Accuracy: 0.5616
Epoch 19/20 | Loss: 1.1009 | Accuracy: 0.5672
Epoch 20/20 | Loss: 1.0826 | Accuracy: 0.5768


accuracy,▁▂▄▄▅▅▅▆▆▆▆▇▇▇▇▇████
loss,█▆▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
accuracy,0.57679
loss,1.08259


In [21]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Accuracy: {correct / total:.4f}")


In [22]:
evaluate(model, val_loader)

Validation Accuracy: 0.5998
